<a href="https://colab.research.google.com/github/TIANBOQIU/AppliedDeepLearning/blob/master/HW2_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tf-nightly-2.0-preview

    100% |████████████████████████████████| 78.9MB 612kB/s 
    100% |████████████████████████████████| 61kB 21.4MB/s 
    100% |████████████████████████████████| 3.2MB 11.2MB/s 
    100% |████████████████████████████████| 358kB 20.4MB/s 


In [16]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import numpy as np
import zipfile

from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50, InceptionV3

print(tf.__version__)

1.13.0-rc1


In [26]:
# Data
# Recorded videos for 3 landmarks, then convert them into images
# -classes: 'AlmaMater', 'Lion', 'Thinker'     
# then split into /train 100 each  /val 50 each  /test 25 each

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [36]:
base_dir = 'drive/My Drive/DeepLearning/datasets/ColumbiaStatues'
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')
test_dir = os.path.join(base_dir,'test')

train_paths = [os.path.join(train_dir, i) for i in os.listdir(train_dir)]
val_paths = [os.path.join(val_dir, i) for i in os.listdir(val_dir)]
test_paths = [os.path.join(test_dir, i) for i in os.listdir(test_dir)]

n_train_per_class = [len(os.listdir(i)) for i in train_paths]
n_val_per_class = [len(os.listdir(i)) for i in val_paths]
n_test_per_class = [len(os.listdir(i)) for i in test_paths]
print('train:',n_train_per_class)
print('val:',n_val_per_class)
print('test:',n_test_per_class)
n_classes = len(os.listdir(train_dir))
print('n_classes', n_classes)

train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)
BATCH_SIZE=20
TARGET_SHAPE=150
train_data = train_data_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(TARGET_SHAPE,TARGET_SHAPE),
                                               class_mode='categorical')
val_data = val_data_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=val_dir,
                                               shuffle=True,
                                               target_size=(TARGET_SHAPE,TARGET_SHAPE),
                                               class_mode='categorical')
test_data = test_data_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=test_dir,
                                               shuffle=True,
                                               target_size=(TARGET_SHAPE,TARGET_SHAPE),
                                               class_mode='categorical')



train: [100, 100, 100]
val: [50, 50, 50]
test: [25, 25, 25]
n_classes 3
Found 300 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 75 images belonging to 3 classes.


In [37]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step


In [38]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [0]:
def extract_features_fromVGG16(data_from_gen, n):
  BATCH_SIZE=data_from_gen.batch_size
  n_classes = len(data_from_gen.class_indices)
  features = np.zeros(shape=(n,4,4,512))
  labels = np.zeros(shape=(n, n_classes))
  
  i = 0
  for images_batch, labels_batch in data_from_gen:
    features[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = conv_base.predict(images_batch)
    labels[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = labels_batch
    print('Bacth{}/{} Done'.format(i+1, int(n/BATCH_SIZE)))
    i += 1
    if i * BATCH_SIZE >= n:
      break
  return features, labels


In [62]:
train_features, train_labels = extract_features_fromVGG16(train_data, sum(n_train_per_class))

Bacth1/15 Done
Bacth2/15 Done
Bacth3/15 Done
Bacth4/15 Done
Bacth5/15 Done
Bacth6/15 Done
Bacth7/15 Done
Bacth8/15 Done
Bacth9/15 Done
Bacth10/15 Done
Bacth11/15 Done
Bacth12/15 Done
Bacth13/15 Done
Bacth14/15 Done
Bacth15/15 Done


In [64]:
VAL_BATCH_SIZE = 30
val_data = val_data_gen.flow_from_directory(batch_size=VAL_BATCH_SIZE,
                                               directory=val_dir,
                                               shuffle=True,
                                               target_size=(TARGET_SHAPE,TARGET_SHAPE),
                                               class_mode='categorical')


val_features, val_labels = extract_features_fromVGG16(val_data, sum(n_val_per_class))

Found 150 images belonging to 3 classes.
Bacth1/5 Done
Bacth2/5 Done
Bacth3/5 Done
Bacth4/5 Done
Bacth5/5 Done


In [65]:
TEST_BATCH_SIZE = 25
test_data = test_data_gen.flow_from_directory(batch_size=TEST_BATCH_SIZE,
                                               directory=test_dir,
                                               shuffle=True,
                                               target_size=(TARGET_SHAPE,TARGET_SHAPE),
                                               class_mode='categorical')


test_features, test_labels = extract_features_fromVGG16(test_data, sum(n_test_per_class))

Found 75 images belonging to 3 classes.
Bacth1/3 Done
Bacth2/3 Done
Bacth3/3 Done


In [0]:
# Model
train_features = np.reshape(train_features, (sum(n_train_per_class),4*4*512))
val_features = np.reshape(val_features, (sum(n_val_per_class), 4*4*512))
test_features = np.reshape(test_features, (sum(n_test_per_class), 4*4*512))

In [69]:
print('feature_shape','label_shape')
print(train_features.shape, train_labels.shape)
print(val_features.shape, val_labels.shape)
print(test_features.shape, test_labels.shape)

feature_shape label_shape
(300, 8192) (300, 3)
(150, 8192) (150, 3)
(75, 8192) (75, 3)


In [70]:
model=Sequential()
model.add(Dense(256, activation='relu', input_dim=4*4*512))
model.add(keras.layers.Dropout(0.2))
model.add(Dense(3, activation='sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
history = model.fit(train_features, train_labels, epochs=30,batch_size=20,validation_data=(val_features, val_labels))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 300 samples, validate on 150 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
300/300 [==============================] - 1s 2ms/sample - loss: 0.1636 - acc: 0.9267 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 2/30
300/300 [==============================] - 0s 406us/sample - loss: 7.2770e-04 - acc: 1.0000 - val_loss: 4.1573e-06 - val_acc: 1.0000
Epoch 3/30
300/300 [==============================] - 0s 420us/sample - loss: 1.5179e-05 - acc: 1.0000 - val_loss: 3.9776e-07 - val_acc: 1.0000
Epoch 4/30
300/300 [==============================] - 0s 430us/sample - loss: 1.3099e-05 - acc: 1.0000 - val_loss: 1.8279e-07 - val_acc: 1.0000
Epoch 5/30
300/300 [==============================] - 0s 419us/sample - loss: 9.8112e-07 - acc: 1.0000 - val_loss: 1.7206e-07 - val_acc: 1.0000
Epoch 6/30
300/300 [==============================] - 0s 431us/sample - loss: 3.3

In [120]:
# TEST ACCURACY
test_pred = np.array(model.predict(test_features))
test_pred = [np.argmax(i) for i in test_pred]
len(test_pred)
test_labels_num = [np.argmax(i) for i in test_labels]
len(test_labels_num)
test_acc = sum(np.equal(test_pred, test_labels_num)) / len(test_pred)
print('TEST accuracy:', test_acc)

TEST accuracy: 1.0


**The accuracy of the model is 100% when using VGG16 as the base model for transfer learning.**

##Use other pretrained model

**try to acquire  a model as small as possible**

In [126]:
from keras.applications import MobileNetV2

conv_base = MobileNetV2(input_shape=(160,160,3), alpha=1.0, include_top=False,weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


In [127]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [18]:
! ls $train_dir

AlmaMater  Lion  Thinker
